In [ ]:
import json
import os
from pathlib import Path

from loguru import logger

from src.Dto.keywords import URL
from src.Dto.parameter import EnumParam
from src.ca import RuntimeInfoManager
from src.languagemodel.LanguageModel import ParamValueModel, ResponseModel
from src.openapiParser import Parser

swagger = "/Users/naariah/Documents/Python_Codes/RestCT/exp/swagger/GitLab/Branch.json"
os.environ["swagger"] = swagger
os.environ["model"] = "gpt-3.5-turbo-1106"
os.environ["language_model_key"] = ""
# os.environ["http_proxy"] = "http://localhost:7890"
# os.environ["https_proxy"] = "http://localhost:7890"

with Path(swagger).open("r") as fp:
    spec = json.load(fp)

p = Parser(logger)
p.parse()
op = p.operations[1]
ep = [p for p in op.parameterList if not isinstance(p, EnumParam)]
bp = ep[0]
pl = bp.seeAllParameters()
p0 = pl[0].getGlobalName()
definition = spec.get("definitions").copy()
p_l = spec["paths"][op.url.replace(URL.baseurl, "")][op.method.value]["parameters"]
plstr = json.dumps(p_l)


# ref = str()
# for p_i in p_l:
#     if p_i["name"] == bp.name:
#         ref = p_i["schema"].get("$ref").split("/")[-1]
# def_dict = definition[ref]


def get_info(param, definition, def_dict, body):
    for p_n in param.split("@"):
        if p_n == body.name:
            continue
        elif p_n == "_item":
            ref = def_dict.get("items").get("$ref").split("/")[-1]
            def_dict = definition.get(ref)
        else:
            info = def_dict.get("properties").get(p_n)
            def_dict = info
    return info


# i = get_info(p0, definition, def_dict, bp)
# print(i)

r = RuntimeInfoManager()

In [ ]:
res = [
    (400, {"message": {"import_url": ["is blocked: Only allowed schemes are http, https, git"], "name": [],
                       "limit_reached": []}}),
    (400, {"error": "name, path are missing, at least one parameter must be provided"}),
    (400, {"error": "avatar is invalid"}),
    (400, {"message": {"namespace": ["is not valid"], "limit_reached": []}}),
    (400, {"message": {"project_feature.merge_requests_access_level": [
        "cannot have higher visibility level than repository access level"], "project_feature.builds_access_level": [
        "cannot have higher visibility level than repository access level"],
        "import_url": ["is blocked: Only allowed schemes are http, https, git"],
        "repository_storage": ["is not included in the list"], "name": [], "limit_reached": []}})
]


In [ ]:
pv = ParamValueModel(op, ep, r, "/Users/naariah/Experiment/LLM")
rm = ResponseModel(op, r, "/Users/naariah/Experiment/LLM", res)
pt = pv.build_prompt()
a = rm.execute()
print()
print(r.get_llm_constrainted_params(op))
# print(pt)
# a = pv.execute()
# print()
# t = r.get_llm_examples().get(op)
# print()

In [ ]:
text1 = "Error: Internal Error: Unrecognized token 'AkGlSsinrjah6LxetN8K': was expecting (JSON String, Number, Array, Object or token 'null', 'true' or 'false') at [Source: (String)'AkGlSsinrjah6LxetN8K'; line: 1, column: 21]"
text2 = "Error: Internal Error: Unrecognized token 'Nps': was expecting (JSON String, Number, Array, Object or token 'null', 'true' or 'false') at [Source: (String)'Nps'; line: 1, column: 4]"

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(string1, string2):
    # 将两个字符串组合成一个列表
    documents = [string1, string2]

    # 使用 CountVectorizer 转换文本为词频矩阵
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)

    # 使用 cosine_similarity 计算余弦相似度
    cosine_sim = cosine_similarity(sparse_matrix, sparse_matrix)

    # 获取相似度值
    similarity_value = cosine_sim[0][1]

    return similarity_value

similarity = calculate_cosine_similarity(text1, text2)
print(f"Cosine Similarity: {similarity}")